<a href="https://colab.research.google.com/github/StarkTech-3000/Luvishka_Curtin/blob/main/Luvishka_Starter_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [189]:
!pip install plotly ipywidgets requests pandas -quiet


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [200]:
! pip install pio

In [201]:
import requests
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime

def get_weather_forecast(location, api_key):
    """Fetch 5-day forecast and return a DataFrame plus a location string."""
    # 1) Geocode
    geo = requests.get(
        "http://api.openweathermap.org/geo/1.0/direct",
        params={"q": location, "limit": 1, "appid": api_key}
    ).json()
    if not geo:
        raise ValueError(f"Location not found: {location}")
    lat, lon = geo[0]["lat"], geo[0]["lon"]
    city, country = geo[0]["name"], geo[0].get("country","")

    # 2) Forecast
    fc = requests.get(
        "https://api.openweathermap.org/data/2.5/forecast",
        params={"lat": lat, "lon": lon, "units": "metric", "appid": api_key}
    ).json()

    records = []
    for e in fc["list"]:
        dt = datetime.fromtimestamp(e["dt"])
        records.append({
            "date": dt.strftime("%a %d %b"),
            "temp": e["main"]["temp"],
            "humidity": e["main"]["humidity"],
            "wind": e["wind"]["speed"],
            "desc": e["weather"][0]["description"].title()
        })
    df = pd.DataFrame(records)
    return df, f"{city}, {country}"

def show_plotly_dashboard(df, loc_name):
    # aggregate by date
    df_day = df.groupby("date").agg({
        "temp":"mean", "humidity":"mean", "wind":"mean"
    }).reset_index()

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=df_day["date"], y=df_day["temp"],
        name="Avg Temp (°C)", marker_color="lightskyblue"
    ))
    fig.add_trace(go.Scatter(
        x=df_day["date"], y=df_day["humidity"],
        name="Avg Humidity (%)", yaxis="y2", marker_color="seagreen"
    ))
    fig.update_layout(
        title=f"📍 {loc_name} — 5-Day Forecast",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Temperature (°C)"),
        yaxis2=dict(title="Humidity (%)", overlaying="y", side="right"),
        template="plotly_white", height=450
    )
    fig.show()

    print("\nDaily Summaries:")
    for d in df_day["date"]:
        row = df_day[df_day["date"]==d].iloc[0]
        print(f"{d}: {row.temp:.1f}°C, {row.humidity:.0f}% hum, {row.wind:.1f} m/s")

# === Main ===
print("=== Weather Dashboard ===")
apikey = input("OpenWeatherMap API Key: ").strip()
loc = input("Location (e.g. Paris, FR): ").strip()
try:
    df, name = get_weather_forecast(loc, apikey)
    show_plotly_dashboard(df, name)
except ValueError as ve:
    print("❌", ve)
except requests.HTTPError as he:
    print("HTTP error:", he)
except Exception as e:
    print("Error:", e)


=== Weather Dashboard ===
OpenWeatherMap API Key: 4368d90223145a8c8f649a01a21440bc
Location (e.g. Paris, FR): Paris



Daily Summaries:
Mon 26 May: 15.6°C, 57% hum, 3.2 m/s
Sat 24 May: 14.3°C, 66% hum, 2.6 m/s
Sun 25 May: 16.5°C, 73% hum, 4.8 m/s
Tue 27 May: 16.7°C, 65% hum, 3.7 m/s
Wed 28 May: 17.2°C, 67% hum, 4.9 m/s
